In [0]:
peopleDF = spark.read.parquet("/FileStore/tables/people-10m.parquet")
peopleDF.show(5)

+---+---------+----------+----------+------+-------------------+-----------+------+
 id|firstName|middleName| lastName|gender| birthDate| ssn|salary|
+---+---------+----------+----------+------+-------------------+-----------+------+
 1| Pennie| Carry|Hirschmann| F|1955-07-02 04:00:00|981-43-9345| 56172|
 2| An| Amira| Cowper| F|1992-02-08 05:00:00|978-97-8086| 40203|
 3| Quyen| Marlen| Dome| F|1970-10-11 04:00:00|957-57-8246| 53417|
 4| Coralie| Antonina| Marshal| F|1990-04-11 04:00:00|963-39-4885| 94727|
 5| Terrie| Wava| Bonar| F|1980-01-16 05:00:00|964-49-8051| 79908|
+---+---------+----------+----------+------+-------------------+-----------+------+
only showing top 5 rows

In [0]:
name_lst = [c for c in peopleDF.columns if 'Name' in c]
print(name_lst)
nameDF = peopleDF[name_lst]
nameDF.show(1)

nameDF.groupBy('middleName').count().show(5)

['firstName', 'middleName', 'lastName']
+---------+----------+----------+
firstName|middleName| lastName|
+---------+----------+----------+
 Pennie| Carry|Hirschmann|
+---------+----------+----------+
only showing top 1 row

+----------+-----+
middleName|count|
+----------+-----+
 Tegan| 1309|
 Reta| 1277|
 Julianne| 1260|
 Sandi| 1233|
 Susanna| 1255|
+----------+-----+
only showing top 5 rows

In [0]:
firstName = peopleDF.select('firstName').distinct()
print(firstName.count())
sortingDF = firstName.orderBy('firstName', ascending=False)
sortingDF.show(5)

5113
+---------+
firstName|
+---------+
 Zulma|
 Zulema|
 Zula|
 Zoraida|
 Zora|
+---------+
only showing top 5 rows

In [0]:
from pyspark.sql.functions import year, col

In [0]:
dordonDF = peopleDF.\
            select(year('birthDate').alias("birthYear"), "firstName").\
            filter((col("firstName")=='Donna') | (col("firstName")=='Dorothy')).\
            filter("gender == 'F'").\
            groupBy('birthYear','firstName').\
            count().orderBy('birthYear').show(5)

+---------+---------+-----+
birthYear|firstName|count|
+---------+---------+-----+
 1952| Donna| 24|
 1952| Dorothy| 21|
 1953| Donna| 37|
 1953| Dorothy| 38|
 1954| Dorothy| 29|
+---------+---------+-----+
only showing top 5 rows

In [0]:
dordonDF = peopleDF.\
            select(year('birthDate').alias("birthYear"), "firstName").\
            filter((col("firstName")=='Donna') | (col("firstName")=='Dorothy')).\
            filter("gender == 'F'").\
            filter(year('birthDate') >= "1990").\
            groupBy('birthYear','firstName').\
            count().orderBy('birthYear')

In [0]:
display(dordonDF)

birthYear,firstName,count
1990,Donna,24
1990,Dorothy,29
1991,Dorothy,25
1991,Donna,27
1992,Dorothy,39
1992,Donna,30
1993,Donna,22
1993,Dorothy,28
1994,Donna,26
1994,Dorothy,21


In [0]:
valuesA = [('tom', 70), ('bob', 100), ('sam', 60), ('lee', 85), ('jun', 75)]
valuesB = [('tom', 55), ('sam', 88), ('kim', 90), ('jun', 67), ('choi', 95)]
mathS_df = spark.createDataFrame(valuesA, ['name', 'score'])
englishS_df = spark.createDataFrame(valuesB, ['name', 'score'])

In [0]:
mathS_df.show()
englishS_df.show()

+----+-----+
name|score|
+----+-----+
 tom| 70|
 bob| 100|
 sam| 60|
 lee| 85|
 jun| 75|
+----+-----+

+----+-----+
name|score|
+----+-----+
 tom| 55|
 sam| 88|
 kim| 90|
 jun| 67|
choi| 95|
+----+-----+

In [0]:
mathS_df = mathS_df.withColumnRenamed('score', 'math score')
englishS_df = englishS_df.withColumnRenamed('score', 'english score')

join1 = mathS_df.join(englishS_df, mathS_df.name == englishS_df.name)
join1.show()

+----+----------+----+-------------+
name|math score|name|english score|
+----+----------+----+-------------+
 jun| 75| jun| 67|
 tom| 70| tom| 55|
 sam| 60| sam| 88|
+----+----------+----+-------------+

In [0]:
mathS_df = mathS_df.withColumnRenamed('score', 'math score')
englishS_df = englishS_df.withColumnRenamed('score', 'english score')

join2 = mathS_df.join(englishS_df, mathS_df.name == englishS_df.name, how='right')\
                .filter(mathS_df.name.isNull())
join2.show()

+----+----------+----+-------------+
name|math score|name|english score|
+----+----------+----+-------------+
null| null|choi| 95|
null| null| kim| 90|
+----+----------+----+-------------+

In [0]:
from pyspark.sql.functions import count, desc

In [0]:
top10MaleFirstNameDF = peopleDF.\
                        select("firstName").\
                        filter("gender = 'M'").\
                        groupBy("firstName").\
                        agg(count(col('firstName')).alias('total')).\
                        orderBy(desc('total')).\
                        limit(10)

top10MaleFirstNameDF.show(5)

+---------+-----+
firstName|total|
+---------+-----+
 Marcelo| 4197|
 Reynaldo| 4193|
 Jeffrey| 4177|
 Sammie| 4170|
 Troy| 4167|
+---------+-----+
only showing top 5 rows

In [0]:
top10MaleFirstNameDF.createOrReplaceTempView("top10MaleFirstName")
resultsDF = spark.sql("select * from top10MaleFirstName order by FirstName")
display(resultsDF)

firstName,total
Darin,4166
Fritz,4157
Jeffrey,4177
Jess,4153
Marcelo,4197
Reynaldo,4193
Sammie,4170
Tomas,4161
Troy,4167
Wyatt,4159
